# Model Training

In [1]:
import os

In [2]:
%pwd

'/home/corti/Desktop/mlopsProject/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/corti/Desktop/mlopsProject'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path              # The root directory where the model and data are stored
    data_path: Path             # The path to the data for training and evaluation
    model_ckpt: Path            # The path to the model checkpoint
    num_train_epochs: int       # The number of epochs for training
    warmup_steps: int           # The number of steps for the warmup phase
    per_device_train_batch_size: int  # The batch size for training per device
    per_device_eval_batch_size: int   # The batch size for evaluation per device
    weight_decay: float         # The weight decay for the optimizer
    logging_steps: int          # The number of steps between each logging
    evaluation_strategy: str    # The strategy to use for evaluation
    eval_steps: int             # The number of steps between each evaluation
    save_steps: int             # The number of steps between each model saving
    gradient_accumulation_steps: int  # The number of steps to accumulate gradients before performing an optimization step

In [6]:
from ConversationSummarizer.constants import *
from ConversationSummarizer.utils.common import read_yaml, create_directories

In [7]:
# Define a class for managing configurations
class ConfigurationManager:
    # Initialize the ConfigurationManager with paths to the configuration and parameters files
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Read the configuration and parameters from the specified files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create the directory for storing artifacts
        create_directories([self.config.artifacts_root])

    # Method to get the configuration for the model trainer
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        # Get the model trainer configuration and parameters
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        # Create the root directory for the model trainer
        create_directories([config.root_dir])

        # Create a ModelTrainerConfig object with the specified configuration and parameters
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        # Return the ModelTrainerConfig object
        return model_trainer_config

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

/home/corti/anaconda3/envs/ConvSum/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-05 16:04:40,229: INFO: config: PyTorch version 2.2.0 available.]


In [9]:
# Define a class for training the model
class ModelTrainer:
    # Initialize the ModelTrainer with a ModelTrainerConfig object
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    # Method to train the model
    def train(self):
        # Determine the device to use for training
        device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load the tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)

        # Create a data collator for sequence-to-sequence models
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Load the data for training and evaluation
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # Define the training arguments
        trainer_args = Seq2SeqTrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs, 
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps, 
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        ) 

        # Initialize the trainer
        trainer = Seq2SeqTrainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        # Train the model
        trainer.train()

        # Save the trained model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-02-05 16:04:40,575: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-05 16:04:40,579: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 16:04:40,580: INFO: common: created directory at: artifacts]
[2024-02-05 16:04:40,581: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2024-02-05 16:04:49,205: ERROR: jupyter: Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.]


wandb: Currently logged in as: nicolacortinovis98 (cepp_ml). Use `wandb login --relogin` to force relogin


  0%|          | 0/920 [00:00<?, ?it/s]

KeyboardInterrupt: 